In [1]:
import pandas as pd
import numpy as np
import random

def set_random_seed(seed):
    # Python의 기본 랜덤 시드 고정
    random.seed(seed)

# 시드 설정
set_random_seed(42)


In [7]:
PRE_PATH = '/upstage-cv-classification-cv2/data/'
TRAIN_CSV_PATH = PRE_PATH + 'train_aug.csv'
VALID_CSV_PATH = PRE_PATH + 'valid.csv'
VALID_37_CSV_PATH = PRE_PATH + 'valid37.csv'

In [8]:
train_df = pd.read_csv(TRAIN_CSV_PATH)
valid_df = pd.read_csv(VALID_CSV_PATH)

train_37_df = train_df[(train_df['target'] == 3) | (train_df['target'] == 7)]
valid_37_df = valid_df[(valid_df['target'] == 3) | (valid_df['target'] == 7)]
valid_37_df.to_csv(VALID_37_CSV_PATH, index = False)


In [9]:
def create_pairs(df):
    class_3 = df[df['target'] == 3].reset_index(drop = True)
    class_7 = df[df['target'] == 7].reset_index(drop = True)
    
    result_df = pd.DataFrame()
    
    positive_pairs_3 = []
    positive_pairs_7 = []
    negative_pairs = []

    # positive pair 3
    for _ in range(1000):
        idx1, idx2 = np.random.choice(len(class_3), 2, replace = False)
        item = {'img1_id' : class_3.iloc[idx1]['ID'],
         'img2_id' : class_3.iloc[idx2]['ID']}
        positive_pairs_3.append(item)

    positive_pairs_3_df = pd.DataFrame(positive_pairs_3)
    positive_pairs_3_df['target'] = 1

    # positive pair 7
    for _ in range(1000):
        idx1, idx2 = np.random.choice(len(class_7), 2, replace = False)
        item = {'img1_id' : class_7.iloc[idx1]['ID'],
         'img2_id' : class_7.iloc[idx2]['ID']}
        positive_pairs_7.append(item)

    positive_pairs_7_df = pd.DataFrame(positive_pairs_7)
    positive_pairs_7_df['target'] = 1

    # negative pair 7 - 3
    for _ in range(1000):
        idx1 = np.random.choice(len(class_7))
        idx2 = np.random.choice(len(class_3))
        item = {'img1_id' : class_7.iloc[idx1]['ID'],
         'img2_id' : class_3.iloc[idx2]['ID']}
        negative_pairs.append(item)

    # negative pair 3 - 7
    for _ in range(1000):
        idx1 = np.random.choice(len(class_3))
        idx2 = np.random.choice(len(class_7))
        item = {'img1_id' : class_3.iloc[idx1]['ID'],
         'img2_id' : class_7.iloc[idx2]['ID']}
        negative_pairs.append(item)

    negative_pairs_df = pd.DataFrame(negative_pairs)
    negative_pairs_df['target'] = -1

    result_df = pd.concat([positive_pairs_3_df, positive_pairs_7_df, negative_pairs_df], axis = 0)
    return result_df

pairs_df = create_pairs(train_37_df)
pairs_df



,img1_id,img2_id,target
0,rotate_00078da144ebc783909.jpg,rotate_090308ffb1480b710bb.jpg,1
1,rotate_27005c30f3c5be4bdcb.jpg,rotate_270aec62dced7af97cd.jpg,1
2,rotate_1357bc3f5f4cc4f1873.jpg,roate_shift_noise_27009a930520afa0ab9.jpg,1
3,roate_shift_noise_3156bd9b10df1a9e223.jpg,rotate_2257c42f2261aedc210.jpg,1
4,roate_shift_noise_2257b6175b967d7c92f.jpg,roate_shift_noise_270d4dcb750454d78a4.jpg,1
...,...,...,...
1995,roate_shift_noise_225df7d7932966d7e22.jpg,rotate_1353b2ddc667cea0d48.jpg,-1
1996,roate_shift_noise_00024204bdb434192db.jpg,roate_shift_noise_2702b04e4a315457bc0.jpg,-1
1997,rotate_1803f0cca298c59752d.jpg,rotate_090a376fbdb67bc4a92.jpg,-1
1998,rotate_225d41f82a2e58ebf33.jpg,rotate_1805403da8f45eaf06c.jpg,-1


In [10]:
pairs_df.to_csv('/upstage-cv-classification-cv2/data/contra_train.csv', index=False)